In [29]:
# !wget https://raw.githubusercontent.com/mxw/grmr/master/src/finaltests/bible.txt

--2023-12-22 22:35:27--  https://raw.githubusercontent.com/mxw/grmr/master/src/finaltests/bible.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4451368 (4.2M) [text/plain]
Saving to: ‘bible.txt.1’

bible.txt.1           0%[                    ]       0  --.-KB/s               

bible.txt.1         100%[===================>]   4.25M  --.-KB/s    in 0.09s   

2023-12-22 22:35:28 (49.1 MB/s) - ‘bible.txt.1’ saved [4451368/4451368]



In [30]:
with open("bible.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [31]:
print(len(text))

4351186


In [32]:
print(text[:1000])

1:1 In the beginning God created the heaven and the earth.

1:2 And the earth was without form, and void; and darkness was upon
the face of the deep. And the Spirit of God moved upon the face of the
waters.

1:3 And God said, Let there be light: and there was light.

1:4 And God saw the light, that it was good: and God divided the light
from the darkness.

1:5 And God called the light Day, and the darkness he called Night.
And the evening and the morning were the first day.

1:6 And God said, Let there be a firmament in the midst of the waters,
and let it divide the waters from the waters.

1:7 And God made the firmament, and divided the waters which were
under the firmament from the waters which were above the firmament:
and it was so.

1:8 And God called the firmament Heaven. And the evening and the
morning were the second day.

1:9 And God said, Let the waters under the heaven be gathered together
unto one place, and let the dry land appear: and it was so.

1:10 And God called the d

In [33]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !"$%'()*,-./0123456789:;?@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
80


In [34]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("Test"))
print(decode(encode("Test")))

[47, 58, 72, 73]
Test


In [35]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([4351186]) torch.int64
tensor([15, 24, 15,  1, 36, 67,  1, 73, 61, 58,  1, 55, 58, 60, 62, 67, 67, 62,
        67, 60,  1, 34, 68, 57,  1, 56, 71, 58, 54, 73, 58, 57,  1, 73, 61, 58,
         1, 61, 58, 54, 75, 58, 67,  1, 54, 67, 57,  1, 73, 61, 58,  1, 58, 54,
        71, 73, 61, 12,  0,  0, 15, 24, 16,  1, 28, 67, 57,  1, 73, 61, 58,  1,
        58, 54, 71, 73, 61,  1, 76, 54, 72,  1, 76, 62, 73, 61, 68, 74, 73,  1,
        59, 68, 71, 66, 10,  1, 54, 67, 57,  1, 75, 68, 62, 57, 25,  1, 54, 67,
        57,  1, 57, 54, 71, 64, 67, 58, 72, 72,  1, 76, 54, 72,  1, 74, 69, 68,
        67,  0, 73, 61, 58,  1, 59, 54, 56, 58,  1, 68, 59,  1, 73, 61, 58,  1,
        57, 58, 58, 69, 12,  1, 28, 67, 57,  1, 73, 61, 58,  1, 46, 69, 62, 71,
        62, 73,  1, 68, 59,  1, 34, 68, 57,  1, 66, 68, 75, 58, 57,  1, 74, 69,
        68, 67,  1, 73, 61, 58,  1, 59, 54, 56, 58,  1, 68, 59,  1, 73, 61, 58,
         0, 76, 54, 73, 58, 71, 72, 12,  0,  0, 15, 24, 17,  1, 28, 67, 57,  1,
      

In [36]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [37]:
block_size = 8
train_data[:block_size+1]

tensor([15, 24, 15,  1, 36, 67,  1, 73, 61])

In [38]:
x = train_data[:block_size]
y = train_data[1 : block_size + 1]
for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([15]) the target: 24
when input is tensor([15, 24]) the target: 15
when input is tensor([15, 24, 15]) the target: 1
when input is tensor([15, 24, 15,  1]) the target: 36
when input is tensor([15, 24, 15,  1, 36]) the target: 67
when input is tensor([15, 24, 15,  1, 36, 67]) the target: 1
when input is tensor([15, 24, 15,  1, 36, 67,  1]) the target: 73
when input is tensor([15, 24, 15,  1, 36, 67,  1, 73]) the target: 61


In [39]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x,y

xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)
print("----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[10,  1, 62, 59,  1, 73, 61, 58],
        [20,  1, 47, 61, 58, 71, 58, 59],
        [68,  1, 66, 78,  1, 72, 68, 67],
        [60,  1, 73, 68,  1, 28, 54, 71]])
targets:
torch.Size([4, 8])
tensor([[ 1, 62, 59,  1, 73, 61, 58,  1],
        [ 1, 47, 61, 58, 71, 58, 59, 68],
        [ 1, 66, 78,  1, 72, 68, 67,  1],
        [ 1, 73, 68,  1, 28, 54, 71, 68]])
----
when input is [10] the target: 1
when input is [10, 1] the target: 62
when input is [10, 1, 62] the target: 59
when input is [10, 1, 62, 59] the target: 1
when input is [10, 1, 62, 59, 1] the target: 73
when input is [10, 1, 62, 59, 1, 73] the target: 61
when input is [10, 1, 62, 59, 1, 73, 61] the target: 58
when input is [10, 1, 62, 59, 1, 73, 61, 58] the target: 1
when input is [20] the target: 1
when input is [20, 1] the target: 47
when input is [20, 1, 47] the target: 61
when input is [20, 1, 47, 61] the target: 58
when input is [20, 1, 47, 61, 58] the target: 71
when input is [20, 1, 47, 6

In [40]:
print(xb)

tensor([[10,  1, 62, 59,  1, 73, 61, 58],
        [20,  1, 47, 61, 58, 71, 58, 59],
        [68,  1, 66, 78,  1, 72, 68, 67],
        [60,  1, 73, 68,  1, 28, 54, 71]])


In [41]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off of the logits for the nex token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor or integers
        logits = self.token_embedding_table(idx)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indicies in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 80])
tensor(4.9457, grad_fn=<NllLossBackward0>)

RPYzZ?,3T/79PFY1CMMK9p!"YANd,3@htpf,5tM.U9!BJ0)!B(DdNc;P4/6HM2FabwCPJNFYBfJ)sb'F)YJoEIg!mnV-7B:!x5v1


In [42]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [43]:
# batch_size = 32
# for steps in range(10000):
#     xb, yb = get_batch("train")

#     logits, loss = m(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()

# print(loss.item())

In [44]:
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


cgK9PW*21m*hyGVMuY1xX'0ZI49Jo2uKx;BneXGd!"8sV1dU.3WNF/l kPNtXZTsHb"X*TNBXNlMVrG*:8 MK'Jxog
uY5b'"kIC


In [45]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [46]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, : t + 1]
        xbow[b, t] = torch.mean(xprev, 0)

In [47]:
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [48]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [49]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x  # (B, T, T) @ (B, T, C) ---> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [53]:
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [55]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

True

In [50]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [51]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print("a=")
print(a)
print("--")
print("b=")
print(b)
print("--")
print("c=")
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
